In [ ]:
from __future__ import division

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import statsmodels.api as sm
import time
import scipy
from statsmodels.stats.multitest import fdrcorrection
from statsmodels.stats import multitest
from collections import Counter
import heapq
import statsmodels.api as sm

import os
import math

In [ ]:
def adj_r2_score_and_r2_score(clf, X, y):
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = r2_score(y, clf.predict(X))
    return [1 - (1 - r_squared) * ((n - 1) / (n - p - 1)), r_squared]


def mse(clf, X, y):
    return mean_squared_error(y, clf.predict(X))

def rmse(clf, X, y):
    mse = mean_squared_error(y, clf.predict(X))
    return math.sqrt(mse)    

def coef_se(clf, X, y):
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)

def coef_tval(clf, X, y):
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)



def coef_pval(clf, X, y):

    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p


def residuals(clf, X, y, r_type='standardized'):

    # Make sure value of parameter 'r_type' is one we recognize
    assert r_type in ('raw', 'standardized', 'studentized'), (
        "Invalid option for 'r_type': {0}".format(r_type))
    y_true = y.view(dtype='float')
    # Use classifier to make predictions
    y_pred = clf.predict(X)
    # Make sure dimensions agree (Numpy still allows subtraction if they don't)
    assert y_true.shape == y_pred.shape, (
        "Dimensions of y_true {0} do not match y_pred {1}".format(y_true.shape,
                                                                  y_pred.shape))
    # Get raw residuals, or standardized or standardized residuals
    resids = y_pred - y_true
    if r_type == 'standardized':
        resids = resids / np.std(resids)
    elif r_type == 'studentized':
        # Prepare a blank array to hold studentized residuals
        studentized_resids = np.zeros(y_true.shape[0], dtype='float')
        # Calcluate hat matrix of X values so you can get leverage scores
        hat_matrix = np.dot(
            np.dot(X, np.linalg.inv(np.dot(np.transpose(X), X))),
            np.transpose(X))
        # For each point, calculate studentized residuals w/ leave-one-out MSE
        for i in range(y_true.shape[0]):
            # Make a mask so you can calculate leave-one-out MSE
            mask = np.ones(y_true.shape[0], dtype='bool')
            mask[i] = 0
            loo_mse = np.average(resids[mask] ** 2, axis=0)  # Leave-one-out MSE
            # Calculate studentized residuals
            studentized_resids[i] = resids[i] / np.sqrt(
                loo_mse * (1 - hat_matrix[i, i]))
        resids = studentized_resids
    return resids


def f_squared(clf, X, y):

    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return r_squared  / (1 - r_squared)

def f_stat(clf, X, y):
    """Calculate summary F-statistic for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic value.
    """
    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    return (r_squared / p) / ((1 - r_squared) / (n - p - 1))


def f_stat_pvalue(clf, X, y):
    """Calculate summary F-statistic p value for beta coefficients.

    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].

    Returns
    -------
    float
        The F-statistic p value.
    """
    n = X.shape[0]
    p = X.shape[1]
    r_squared = metrics.r2_score(y, clf.predict(X))
    
    return np.round(scipy.stats.f.sf(f_stat(clf, X, y), n, (n - p - 1)), 15)

def compute_f_statistics(clf, X, y):
    return [f_stat(clf, X, y), f_stat_pvalue(clf, X, y)]





    
    



def summary_validation(X_, y_, opt_Ridge_ = ''):
    
    from statsmodels.stats.multitest import fdrcorrection
    #from statsmodels.stats.multitest import fisher_combined_pvalues
    from scipy.stats import combine_pvalues



    lista_vars = list(X_)
    
    coef_array = np.zeros([len(lista_vars)+1, 10])
    coef_t_value = np.zeros([len(lista_vars)+1, 10])
    coef_p_value = np.zeros([len(lista_vars)+1, 10])

    r2_list =[]
    r2_mean =[]
    f2_list =[]
    f_list =[]
    f_p_value_list =[]
    mse_list =[]
    rmse_list =[]
    
    y_pred_ = []
    y_test_ = []
    for iter_ in tqdm(range(0,10), total=len(range(0, 10)),leave=True, mininterval=1, ascii=True,
                            colour='green', desc='Training model'):
        # Stratified split
        

        X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.20, random_state=iter_)

        model = LinearRegression()

        model.fit(X_train, y_train)

        coef_array[0,iter_] = model.intercept_
        coef_array[1::,iter_] = model.coef_


        coef_t_value[:,iter_] = np.abs(np.round(coef_tval(model, X_test, y_test), 30))
        coef_p_value[:,iter_] = np.round(coef_pval(model, X_test, y_test), 30)


        f2_list.append(f_squared(model, X_test, y_test))
        
        f_p_value_list.append(f_stat_pvalue(model, X_test, y_test))
        
        y_pred_.extend(model.predict(X_test))
        y_test_.extend(y_test)
        
        r2 = r2_score(y_test, model.predict(X_test))
        r2_mean.append(r2)      
    
    n = len(y_pred_)
    p = X_.shape[1]
    r_squared = r2_score(y_test_, y_pred_)
    
      
    r2_list.append(r_squared)
    
    f_list.append((r_squared / p) / ((1 - r_squared) / (n - p - 1)))
    
    mse_list.append(np.round(mean_squared_error(y_test_, y_pred_), 6))
    rmse_list.append(np.round(math.sqrt(mean_squared_error(y_test_, y_pred_)), 6))


    coef_df = pd.DataFrame(
            index=['_intercept'] + lista_vars,
            columns=['Estimate mean','t value mean', 'p value mean', 'p value fdr', 
                     'p value stouffer', 'p value fisher', 
                      'p value tippett', 'Estimate std']
        )
    
    df_coef_value = {}
    df_p_value = {}
    df_t_value = {}
    lista_vars_interc = ['_intercept'] + lista_vars
    for i in range(len(lista_vars_interc)):
        df_coef_value[lista_vars_interc[i]] = coef_array[i,:]
        df_p_value[lista_vars_interc[i]] = coef_p_value[i,:]
        df_t_value[lista_vars_interc[i]] = coef_t_value[i,:]
    
    
    coef_array_mean = np.zeros([len(lista_vars)+1, 1])
    coef_t_value_mean = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_mean = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_fdr = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_stouffer = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_fisher = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_pearson = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_tippett = np.zeros([len(lista_vars)+1, 1])
    coef_p_value_mudholkar_george = np.zeros([len(lista_vars)+1, 1])

    coef_array_std = np.zeros([len(lista_vars)+1, 1])

    coef_p_value = np.where(coef_p_value == 0, 1e-30, coef_p_value)
    
    f_p_value_list_a = np.array(f_p_value_list)
    f_p_value_list_a = np.where(f_p_value_list_a == 0, 1e-30, f_p_value_list_a)
    f_p_value_list = list(f_p_value_list_a)
    
    for i in range(len(lista_vars)+1):
        coef_array_mean[i] = coef_array[i,:].mean()
        coef_t_value_mean[i]= coef_t_value[i,:].mean()
        coef_p_value_mean[i]= coef_p_value[i,:].mean()
        coef_p_value_fdr[i]= fdrcorrection(coef_p_value[i,:], alpha=0.05, is_sorted=False )[1].max()
        coef_p_value_stouffer[i] = combine_pvalues(coef_p_value[i,:], method='stouffer')[1]
        coef_p_value_fisher[i] = combine_pvalues(coef_p_value[i,:], method='fisher')[1]
        coef_p_value_tippett[i] = combine_pvalues(coef_p_value[i,:], method='tippett')[1]

        
        coef_array_std[i] = coef_array[i,:].std()


    coef_df['Estimate mean'] = coef_array_mean
    coef_df['t value mean'] = coef_t_value_mean
    coef_df['p value mean'] = coef_p_value_mean
    coef_df['p value fdr'] = coef_p_value_fdr
    coef_df['p value stouffer'] = coef_p_value_stouffer
    coef_df['p value fisher'] = coef_p_value_fisher
    coef_df['p value tippett'] = coef_p_value_tippett
    coef_df['Estimate std'] = coef_array_std


    empty_str = []
    for i in range(coef_df.shape[0]):
        empty_str.append('')

    coef_df['value'] = empty_str

    coef_df = coef_df.T
    coef_df['R-squared'] = ['','','','','','','', '', np.mean(r2_list)]
    coef_df['CI'] = ['','','','','','','', '', np.std(r2_mean)]
    coef_df['F-squared'] = ['','','','','','','', '', np.mean(f2_list)]
    coef_df['F'] = ['','','','','','','', '', np.mean(f_list)]
    coef_df['mse'] = ['','','','','','','', '', np.mean(mse_list)]
    coef_df['rmse'] = ['','','','','','','', '', np.mean(rmse_list)]  
    coef_df['F-pvalue fisher'] = ['','','','','','','', '', combine_pvalues(f_p_value_list, method='fisher')[1]] 
    
    coef_df['F-pvalue tippett'] = ['','','','','','','', '',combine_pvalues(f_p_value_list, method='tippett')[1]]  

    coef_df = coef_df.T

    
    return [coef_df, df_coef_value, df_p_value, df_t_value, np.mean(mse_list), np.mean(rmse_list), np.mean(r2_list), (f2_list, r2_mean), combine_pvalues(f_p_value_list, method='fisher')[1], np.mean(f_list)]
     
    
def plot_estimate_value_stat_model(regression_model, stat_mod, X_cat_ = [], p_desp = 0.005, title = '',  xlim =[0, 2] , size = 9, GMMSE = -1, stat_model_WO_mmse = 0, pos = [0.95, 0.95] ):
    sns.set(style="whitegrid")
    df = regression_model.reset_index()
    data = df.sort_values('coef', ascending=False)
    
    if(GMMSE>=0):
        inf = data.iloc[:GMMSE, :]
        sup = data.iloc[GMMSE:, :]
        new_file = pd.DataFrame({'Features':['MMSE'],'coef': [0], 'std': [0]})
        data = pd.concat([inf, new_file, sup])
        
        data = data.reset_index()
        #regression_model.index.name = 'Features'
    #display(data)
    f2 = stat_mod[6]  / (1 - stat_mod[6])
    f_prob = '(ns)'
    if(stat_mod[-2]<0.01):
            f_prob = '**'
    elif(stat_mod[-2]<0.05):
            f_prob = '*'
            
          
    color_dict = {}
    color_dict['Age'] = '#026842'
    color_dict['Sex'] = '#f8de7e'
    color_dict['Educ'] = '#006691'
    color_dict['Cat_Educ'] = 'deepskyblue'
    color_dict['MMSE'] = '#9c0720'
    color_dict['Country_cat'] = 'dimgrey'
    color_dict['HDI'] = 'coral'
    color_dict['GINI'] = 'coral'

    bar_color = []
    for i in list(data['Features']):
        bar_color.append(color_dict[i])
    

    
    if(len(X_cat_)>0):
        
        bar_color = get_bar_colors(data, X_cat_)
        
        plt.title(title)
        sns.barplot(x="coef", y="Features", data = data, xerr=data['std'], palette =bar_color)
        plt.xlim(xlim)
    else:
        plt.title(title + '\n' + r'$ R^2($' + str(np.round(stat_mod[6],2))+ ') $F^2($' + str(np.round(f2,2)) + ') ' + ' $F($' + str(np.round(np.mean(stat_mod[-1]),2)) + ') $P($' + f_prob+')', 
                  fontsize=9)
        sns.barplot(x="coef", y="Features", data = data, xerr=data['std'], palette =bar_color)
        plt.xlim(xlim)
        plt.grid(False)

        
    y_step=0  
    for i in range(data.shape[0]):
            
            if(data['p_value'].iloc[y_step]<0.01):
                
                    plt.text(data['coef'].iloc[y_step]+ p_desp, y_step, 
                                     '**',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )
            elif(data['p_value'].iloc[y_step]>= 0.01 and data['p_value'].iloc[y_step]<0.05):
                  
                    plt.text(data['coef'].iloc[y_step]+p_desp, y_step, 
                                     '*',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )  
            else:
                    plt.text(data['coef'].iloc[y_step]+p_desp, y_step, 
                                     ' ',
                                     size= size, rotation=0.,
                                     ha="left", va="center", color = 'black',

                                     )    

            y_step+=1
         
    text_diff =xlim[1]
    
    
    

    if(stat_model_WO_mmse!=0):
        
        f2 = stat_model_WO_mmse[6]  / (1 - stat_model_WO_mmse[6])
        f_prob = '(ns)'
        if(stat_model_WO_mmse[-2]<0.01):
                f_prob = '**'
        elif(stat_model_WO_mmse[-2]<0.05):
            f_prob = '*'
    
        plt.text(xlim[1] - pos[0], df.shape[0]- pos[1],
                 'Without MMSE\n' + r'$ R^2($' + str(np.round(stat_model_WO_mmse[6],2))+ ') $F^2($' + str(np.round(f2,2)) + ') ' + ' \n$F($' + str(np.round(np.mean(stat_model_WO_mmse[-1]),2)) + ') $P($' + f_prob+')',
                                 size= 7, rotation=0.,
                                 ha="right", va="center", color = 'black',
                                 bbox=dict(boxstyle="round",
                                           ec='whitesmoke',
                                            fc='whitesmoke',
                                           )
                                 )
        
       # plt.title(title + '\n' + r'$ R^2($' + str(np.round(stat_mod[6],2))+ ') $F^2($' + str(np.round(f2,2)) + ') ' + ' $F($' + str(np.round(stat_mod.fvalue,2)) + ') $P($' + f_prob+')', 
        #          fontsize=9)

    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)

    plt.xlabel('Estimate', fontsize=8.5)
    plt.ylabel('Features', fontsize=8.5)
    plt.locator_params(axis='x', nbins=4)
    
    if(xlim[1]<0.1):    
        plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))   
    
    
def determinar_intervalo(min_val, max_val, x):


    pasos = np.linspace(min_val, max_val, 7) 
    
    if(max_val == x):
        return 6

    for i in range(7):
        if pasos[i] <= x < pasos[i+1]:
            return (i)
    return None

def rgba_to_hex(rgba):

    return "#{:02X}{:02X}{:02X}".format(rgba[0], rgba[1], rgba[2])



educ_palete = ['#cdffff', '#abe5eb',  '#8acbd8', '#6bb1c6', '#4c98b5', '#2d7fa3', '#006691']
sex_palete = ['#fffff1', '#fcfade',  '#faf5cb', '#f8f0b8', '#f7eaa5', '#f7e492', '#f8de7e']
age_palete = ['#98ffe0', '#80e4c4',  '#69caa8', '#52b18d', '#3b9873', '#23805a', '#026842']
MMSE_palete = ['#ff9ea2', '#f0888b',  '#e07275', '#d05b5f', '#bf4549', '#ae2c34', '#9c0720']

## Sample with cognition

In [ ]:
compl_df_mmse = pd.read_excel('Data/complexity_cognition_supp.xlsx', index_col=0)
compl_df_mmse

In [ ]:
dict_OLS_compl_coef = {}
dict_OLS_compl_R2 = {}

dict_OLS_compl_HDI_coef = {}
dict_OLS_compl_HDI_R2 = {}

dict_OLS_compl_GINI_coef = {}
dict_OLS_compl_GINI_R2 = {}

dict_OLS_compl_Educ_coef = {}
dict_OLS_compl_Educ_R2 = {}

dict_OLS_compl_MMSE_coef = {}
dict_OLS_compl_MMSE_R2 = {}

dict_OLS_compl_Sex_coef = {}
dict_OLS_compl_Sex_R2 = {}

dict_OLS_compl_Age_coef = {}
dict_OLS_compl_Age_R2 = {}

outcomes_list = list(compl_df_mmse.iloc[:,0:-4].columns)

for outcome in outcomes_list:

    y_compl_1A = compl_df_mmse[outcome]
    X_compl_1A = compl_df_mmse[['Age', 'Sex','Educ', 'MMSE']]

    scaler = MinMaxScaler([0.5, 0.95])
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()


    res = summary_validation(X_compl_1A, y_compl_1A)
    table = res[0]
    table = table.iloc[1:-8, 0:-1]
    table = table[['Estimate mean', 't value mean', 'p value stouffer', 'Estimate std']]
    table = table.sort_values(by="Estimate mean", key=lambda x: abs(x), ascending=False)
    print(table.index[0])
    
    if(table.index[0]=='HDI'):
        dict_OLS_compl_HDI_coef[outcome] = res
        dict_OLS_compl_HDI_R2[outcome] = res[6]
    elif(table.index[0]=='GINI'):
        dict_OLS_compl_GINI_coef[outcome] = res
        dict_OLS_compl_GINI_R2[outcome] = res[6]
    elif(table.index[0]=='Educ'):
        dict_OLS_compl_Educ_coef[outcome] = res
        dict_OLS_compl_Educ_R2[outcome] = res[6]
    elif(table.index[0]=='MMSE'):
        dict_OLS_compl_MMSE_coef[outcome] = res
        dict_OLS_compl_MMSE_R2[outcome] = res[6]
    elif(table.index[0]=='Sex'):
        dict_OLS_compl_Sex_coef[outcome] = res
        dict_OLS_compl_Sex_R2[outcome] = res[6]
    elif(table.index[0]=='Age'):
        dict_OLS_compl_Age_coef[outcome] = res
        dict_OLS_compl_Age_R2[outcome] = res[6]

    dict_OLS_compl_coef[outcome] = res
    dict_OLS_compl_R2[outcome] = res[6]
    
    print(outcome, res[6])

In [ ]:
top_10_keys_1A = heapq.nlargest(10, dict_OLS_compl_R2, key=dict_OLS_compl_R2.get)

top_10_keys_HDI = heapq.nlargest(10, dict_OLS_compl_HDI_R2, key=dict_OLS_compl_HDI_R2.get)
top_10_keys_GINI = heapq.nlargest(10, dict_OLS_compl_GINI_R2, key=dict_OLS_compl_GINI_R2.get)
top_10_keys_Educ = heapq.nlargest(10, dict_OLS_compl_Educ_R2, key=dict_OLS_compl_Educ_R2.get)
top_10_keys_MMSE = heapq.nlargest(10, dict_OLS_compl_MMSE_R2, key=dict_OLS_compl_MMSE_R2.get)
top_10_keys_Sex = heapq.nlargest(10, dict_OLS_compl_Sex_R2, key=dict_OLS_compl_Sex_R2.get)
top_10_keys_Age = heapq.nlargest(10, dict_OLS_compl_Age_R2, key=dict_OLS_compl_Age_R2.get)

In [ ]:
top_10_keys_1A[0]

### Figure

In [ ]:
plt.figure(figsize=(7,9))
count=1

feat_for_test_without_mmse = {}
feat_for_test_without_mmse['Age'] = []
feat_for_test_without_mmse['Educ'] = []
feat_for_test_without_mmse['MMSE'] = []

for keys in top_10_keys_Age[0:3]:
    plt.subplot(4,3,count)
    
    model_ = dict_OLS_compl_Age_coef[keys]

    table = model_[0]
    table = table.iloc[1:-8, 0:-1]
    table = table[['Estimate mean', 't value mean', 'p value stouffer', 'Estimate std']]
    table.index.name = 'Features'
    
    table = table.sort_values(by="Estimate mean", key=lambda x: abs(x), ascending=False)
    table = table.rename(columns= {'Estimate std':'std'})
    table = table.rename(columns= {'p value stouffer':'p_value'})
    table = table.rename(columns= {'Estimate mean':'coef'})
    
    table['coef'] = np.abs(table['coef'])

    max_v = table['coef'].max()
    max_v+=max_v*0.5

    if max_v>9 or max_v<3:
        pass
    else:
        max_v = 8
        
    plot_estimate_value_stat_model(table, model_,p_desp =0,  xlim =[0, max_v], title = keys)
    count+=1
    
    feat_for_test_without_mmse['Age'].append(keys)
    
for keys in range(3):
    plt.subplot(4,3,count)
    plt.xlabel('   ')
    plt.ylabel('    ')
    #plt.xticks([])
    #plt.yticks([])
    count+=1
    
for keys in top_10_keys_Educ[0:3]:
    plt.subplot(4,3,count)
    
    model_ = dict_OLS_compl_Educ_coef[keys]

    table = model_[0]
    table = table.iloc[1:-8, 0:-1]
    table = table[['Estimate mean', 't value mean', 'p value stouffer', 'Estimate std']]
    table.index.name = 'Features'
    
    table = table.sort_values(by="Estimate mean", key=lambda x: abs(x), ascending=False)
    table = table.rename(columns= {'Estimate std':'std'})
    table = table.rename(columns= {'p value stouffer':'p_value'})
    table = table.rename(columns= {'Estimate mean':'coef'})
    
    table['coef'] = np.abs(table['coef'])
    max_v = table['coef'].max()
    max_v+=max_v*0.5

    if max_v>9 or max_v<3:
        pass
    else:
        max_v = 8
        
    plot_estimate_value_stat_model(table, model_,p_desp =0,  xlim =[0, max_v], title = keys)
    count+=1
    
    feat_for_test_without_mmse['Educ'].append(keys)
    
for keys in range(3):
    plt.subplot(4,3,count)
    plt.xlabel('   ')
    plt.ylabel('    ')
    #plt.xticks([])
    #plt.yticks([])
    count+=1

count = 7
for keys in top_10_keys_MMSE[0:3]:
    plt.subplot(4,3,count)
    
    model_ = dict_OLS_compl_MMSE_coef[keys]

    table = model_[0]
    table = table.iloc[1:-8, 0:-1]
    table = table[['Estimate mean', 't value mean', 'p value stouffer', 'Estimate std']]
    table.index.name = 'Features'
    
    table = table.sort_values(by="Estimate mean", key=lambda x: abs(x), ascending=False)
    table = table.rename(columns= {'Estimate std':'std'})
    table = table.rename(columns= {'p value stouffer':'p_value'})
    table = table.rename(columns= {'Estimate mean':'coef'})
    
    table['coef'] = np.abs(table['coef'])

    max_v = table['coef'].max()
    max_v+=max_v*0.5
    
    if max_v>9:
        pass
    elif max_v< 0.015:
        max_v = 0.003
    elif max_v< 0.05:
        max_v = 0.05
    else:
        max_v = 2
    
    plot_estimate_value_stat_model(table, model_,p_desp =0,  xlim =[0, max_v], title = keys)
    count+=1
    
    feat_for_test_without_mmse['MMSE'].append(keys)

    
    
plt.tight_layout(pad=1.5);

### Brain plots

In [ ]:
from nilearn import plotting
from nilearn.plotting import plot_glass_brain

In [ ]:
vars_dict = {}
individual_values = []
for i in top_10_keys_MMSE:
    #print(i.split('_')[-2], '\t',np.round(dict_OLS_compl_MMSE_coef[i][6],4))
    if(dict_OLS_compl_MMSE_coef[i][6]>=0.078):
        for j in ['MMSE', 'Sex','Age', 'Educ']:
            vars_dict[i.split('_')[-2] + '_' + j] =[]
            vars_dict[i.split('_')[-2] + '_' + i.split('_')[-1][0].upper() + '_' + j] =[]
            

Rois = []
Rois_lat = []
for i in top_10_keys_MMSE:
    #print(i.split('_')[-2], '\t',np.round(dict_OLS_compl_MMSE_coef[i][6],4))
    if(dict_OLS_compl_MMSE_coef[i][6]>=0.078):
        for j in ['MMSE', 'Sex','Age', 'Educ']:
            table = dict_OLS_compl_MMSE_coef[i][0]
            table = table.iloc[1:-8, 0:-1]
            table = table[['Estimate mean', 't value mean', 'p value stouffer', 'Estimate std']]
            table.index.name = 'Features'
            table = table.rename(columns= {'Estimate std':'std'})
            table = table.rename(columns= {'p value stouffer':'p_value'})
            table = table.rename(columns= {'Estimate mean':'coef'})
            table['coef'] =  np.abs(table['coef'])
            vars_dict[i.split('_')[-2] + '_' + j].append(np.abs(table.loc[j, 'coef']))
            vars_dict[i.split('_')[-2] + '_' + i.split('_')[-1][0].upper() + '_' + j].append(np.abs(table.loc[j, 'coef']))
            Rois.append(i.split('_')[-2])
            Rois_lat.append(i.split('_')[-2] + '_' + i.split('_')[-1][0].upper())
            for z in vars_dict[i.split('_')[-2] + '_' + j]:
                individual_values.append(z)
    
vars_dict

In [ ]:
for i in top_10_keys_Age:   

    if(dict_OLS_compl_Age_coef[i][6]>=0.078):
        for j in ['MMSE', 'Sex','Age', 'Educ']:
            
            if i.split('_')[-2] + '_' + j in list(vars_dict.keys()):
                pass
            else:
                vars_dict[i.split('_')[-2] + '_' + j] =[]
            
            if i.split('_')[-2] + '_' + i.split('_')[-1][0].upper() + '_' + j in list(vars_dict.keys()):
                pass
            else:
                vars_dict[i.split('_')[-2] + '_' + i.split('_')[-1][0].upper() + '_' + j] =[]

            

for i in top_10_keys_Age:
    if(dict_OLS_compl_Age_coef[i][6]>=0.078):
        for j in ['MMSE', 'Sex','Age', 'Educ']:
            table = dict_OLS_compl_Age_coef[i][0]
            table = table.iloc[1:-8, 0:-1]
            table = table[['Estimate mean', 't value mean', 'p value stouffer', 'Estimate std']]
            table.index.name = 'Features'
            table = table.rename(columns= {'Estimate std':'std'})
            table = table.rename(columns= {'p value stouffer':'p_value'})
            table = table.rename(columns= {'Estimate mean':'coef'})
            table['coef'] =  np.abs(table['coef'])
            vars_dict[i.split('_')[-2] + '_' + j].append(np.abs(table.loc[j, 'coef']))
            vars_dict[i.split('_')[-2] + '_' + i.split('_')[-1][0].upper() + '_' + j].append(np.abs(table.loc[j, 'coef']))
            Rois.append(i.split('_')[-2])
            Rois_lat.append(i.split('_')[-2] + '_' + i.split('_')[-1][0].upper())
            for z in vars_dict[i.split('_')[-2] + '_' + j]:
                individual_values.append(z)

unique_Rois = np.unique(Rois)
unique_Rois_lat = np.unique(Rois_lat)
vars_dict

In [ ]:
for i in top_10_keys_Educ:   

    if(dict_OLS_compl_Educ_coef[i][6]>=0.078):
        for j in ['MMSE', 'Sex','Age', 'Educ']:
            
            if i.split('_')[-2] + '_' + j in list(vars_dict.keys()):
                pass
            else:
                vars_dict[i.split('_')[-2] + '_' + j] =[]
            
            if i.split('_')[-2] + '_' + i.split('_')[-1][0].upper() + '_' + j in list(vars_dict.keys()):
                pass
            else:
                vars_dict[i.split('_')[-2] + '_' + i.split('_')[-1][0].upper() + '_' + j] =[]

            

for i in top_10_keys_Educ:
    if(dict_OLS_compl_Educ_coef[i][6]>=0.078):
        for j in ['MMSE', 'Sex','Age', 'Educ']:
            table = dict_OLS_compl_Educ_coef[i][0]
            table = table.iloc[1:-8, 0:-1]
            table = table[['Estimate mean', 't value mean', 'p value stouffer', 'Estimate std']]
            table.index.name = 'Features'
            table = table.rename(columns= {'Estimate std':'std'})
            table = table.rename(columns= {'p value stouffer':'p_value'})
            table = table.rename(columns= {'Estimate mean':'coef'})
            table['coef'] =  np.abs(table['coef'])
            vars_dict[i.split('_')[-2] + '_' + j].append(np.abs(table.loc[j, 'coef']))
            vars_dict[i.split('_')[-2] + '_' + i.split('_')[-1][0].upper() + '_' + j].append(np.abs(table.loc[j, 'coef']))
            Rois.append(i.split('_')[-2])
            Rois_lat.append(i.split('_')[-2] + '_' + i.split('_')[-1][0].upper())
            for z in vars_dict[i.split('_')[-2] + '_' + j]:
                individual_values.append(z)

unique_Rois = np.unique(Rois)
unique_Rois_lat = np.unique(Rois_lat)
vars_dict

In [ ]:
individual_values_MMSE = []
individual_values_Sex = []
individual_values_Age = []
individual_values_Educ = []

for k in vars_dict.keys():
    if 'MMSE' in k:
        for z in vars_dict[k]:
            individual_values_MMSE.append(z)
        
for k in vars_dict.keys():
    if 'Sex' in k:
        for z in vars_dict[k]:
            individual_values_Sex.append(z)
        
for k in vars_dict.keys():
    if 'Age' in k:
        for z in vars_dict[k]:
            individual_values_Age.append(z)
        
for k in vars_dict.keys():
    if 'Educ' in k:
        for z in vars_dict[k]:
            individual_values_Educ.append(z)

### Age

In [ ]:
age_count = {}
age_color = {}


for i in unique_Rois:
    print(i, vars_dict[i + '_Age'])
    
    age_color[i] = np.mean(vars_dict[i + '_Age'])
    age_count[i] = len(vars_dict[i + '_Age'])
    


colormap = plt.cm.Oranges
age_colormap = [colormap(i) for i in np.linspace(0.1, 1, 10)]


init = True
for i in list(age_color.keys()):
    
    if(i.split('_')[0]=='FD' or i.split('_')[0] == 'WMEAN' or i.split('_')[0] == 'PE'):
        if(init):
            display = plot_glass_brain('ROIs/' + 'ALL' + '.nii.gz', 
                                       threshold=20000, display_mode="xz")
            init = False

        
        indx = determinar_intervalo(np.min(individual_values_Age), np.max(individual_values_Age), age_color[i])

        col = age_palete[indx]
        

        display.add_contours('ROIs/' + 'ALL' + '.nii.gz', levels=[3.], 
                             filled=True, alpha = 0.85, colors = col)
        display.add_contours('ROIs/' + 'ALL' + '.nii.gz', levels=[1.], alpha = 1.0, 
                             colors='black', linewidths=age_count[i]/3)

    else:
        if(init):
            display = plot_glass_brain('ROIs/' + i + '.nii.gz', 
                                       threshold=20000, display_mode="xz")
            init = False

        
        indx = determinar_intervalo(np.min(individual_values_Age), np.max(individual_values_Age), age_color[i])

        col = age_palete[indx]
        

        display.add_contours('ROIs/' + i + '.nii.gz', levels=[3.], 
                             filled=True, alpha = 0.85, colors = col)
        display.add_contours('ROIs/' + i + '.nii.gz', levels=[1.], alpha = 1.0, 
                             colors='black', linewidths=age_count[i]/3)
    
    print(i, age_color[i])
    
#display.savefig('Results/Supp/Images/complexity_Age_Brain.png', dpi=300)

### Sex

In [ ]:

sex_count = {}
sex_color = {}


for i in unique_Rois:
    print(i, vars_dict[i + '_Sex'])
    
    sex_color[i] = np.mean(vars_dict[i + '_Sex'])
    sex_count[i] = len(vars_dict[i + '_Sex'])
    


colormap = plt.cm.Oranges
sex_colormap = [colormap(i) for i in np.linspace(0.1, 1, 10)]


init = True
for i in list(sex_color.keys()):
    
    if(i.split('_')[0]=='FD' or i.split('_')[0] == 'WMEAN' or i.split('_')[0] == 'PE'):
        if(init):
            display = plot_glass_brain('ROIs/' + 'ALL' + '.nii.gz', 
                                       threshold=20000, display_mode="xz")
            init = False

        
        indx = determinar_intervalo(np.min(individual_values_Sex), np.max(individual_values_Sex), sex_color[i])

        col = sex_palete[indx]
        

        display.add_contours('ROIs/' + 'ALL' + '.nii.gz', levels=[3.], 
                             filled=True, alpha = 0.85, colors = col)
        display.add_contours('ROIs/' + 'ALL' + '.nii.gz', levels=[1.], alpha = 1.0, 
                             colors='black', linewidths=sex_count[i]/3)

    else:
        if(init):
            display = plot_glass_brain('ROIs/' + i + '.nii.gz', 
                                       threshold=20000, display_mode="xz")
            init = False

        
        indx = determinar_intervalo(np.min(individual_values_Sex), np.max(individual_values_Sex), sex_color[i])

        col = sex_palete[indx]
        

        display.add_contours('ROIs/' + i + '.nii.gz', levels=[3.], 
                             filled=True, alpha = 0.85, colors = col)
        display.add_contours('ROIs/' + i + '.nii.gz', levels=[1.], alpha = 1.0, 
                             colors='black', linewidths=sex_count[i]/3)
    
    print(i, sex_color[i])
    
#display.savefig('Results/Supp/Images/complexity_Sex_Brain.png', dpi=300)

### Cognition

In [ ]:
mmse_count = {}
mmse_color = {}


for i in unique_Rois:
    print(i, vars_dict[i + '_MMSE'])
    
    mmse_color[i] = np.mean(vars_dict[i + '_MMSE'])
    mmse_count[i] = len(vars_dict[i + '_MMSE'])
    


colormap = plt.cm.Reds
mmse_colormap = [colormap(i) for i in np.linspace(0.1, 1, 10)]


init = True
for i in list(mmse_color.keys()):

    if(i.split('_')[0]=='FD' or i.split('_')[0] == 'WMEAN' or i.split('_')[0] == 'PE'):
        if(init):
            display = plot_glass_brain('ROIs/' + 'ALL' + '.nii.gz', 
                                       threshold=20000, display_mode="xz")
            init = False

        
        indx = determinar_intervalo(np.min(individual_values_MMSE), np.max(individual_values_MMSE), mmse_color[i])

        col = MMSE_palete[indx]
        

        display.add_contours('ROIs/' + 'ALL' + '.nii.gz', levels=[3.], 
                             filled=True, alpha = 0.85, colors = col)
        display.add_contours('ROIs/' + 'ALL' + '.nii.gz', levels=[1.], alpha = 1.0, 
                             colors='black', linewidths=mmse_count[i]/3)
    else:
        
        if(init):
            display = plot_glass_brain('ROIs/' + i + '.nii.gz', 
                                       threshold=20000, display_mode="xz")
            init = False

        
        indx = determinar_intervalo(np.min(individual_values_MMSE), np.max(individual_values_MMSE), mmse_color[i])

        col = MMSE_palete[indx]
        

        display.add_contours('ROIs/' + i + '.nii.gz', levels=[3.], 
                             filled=True, alpha = 0.85, colors = col)
        display.add_contours('ROIs/' + i + '.nii.gz', levels=[1.], alpha = 1.0, 
                             colors='black', linewidths=mmse_count[i]/3)
    
    print(i, mmse_color[i])
    
#display.savefig('Results/Supp/Images/complexity_MMSE_Brain.png', dpi=300)

### Education

In [ ]:
educ_count = {}
educ_color = {}


for i in unique_Rois:
    print(i, vars_dict[i + '_Educ'])
    
    educ_color[i] = np.mean(vars_dict[i + '_Educ'])
    educ_count[i] = len(vars_dict[i + '_Educ'])
    


colormap = plt.cm.Blues
educ_colormap = [colormap(i) for i in np.linspace(0.1, 1, 10)]


init = True
for i in list(educ_color.keys()):
    
    if(i.split('_')[0]=='FD' or i.split('_')[0] == 'WMEAN' or i.split('_')[0] == 'PE'):
        if(init):
            display = plot_glass_brain('ROIs/' + 'ALL' + '.nii.gz', 
                                       threshold=20000, display_mode="xz")
            init = False

        
        indx = determinar_intervalo(np.min(individual_values_Educ), np.max(individual_values_Educ), educ_color[i])

        col = educ_palete[indx]
        

        display.add_contours('ROIs/' + 'ALL' + '.nii.gz', levels=[3.], 
                             filled=True, alpha = 0.85, colors = col)
        display.add_contours('ROIs/' + 'ALL' + '.nii.gz', levels=[1.], alpha = 1.0, 
                             colors='black', linewidths=educ_count[i]/3)
    
    else:
        
        if(init):
            display = plot_glass_brain('ROIs/' + i + '.nii.gz', 
                                       threshold=20000, display_mode="xz")
            init = False

        
        indx = determinar_intervalo(np.min(individual_values_Educ), np.max(individual_values_Educ), educ_color[i])

        col = educ_palete[indx]
        

        display.add_contours('ROIs/' + i + '.nii.gz', levels=[3.], 
                             filled=True, alpha = 0.85, colors = col)
        display.add_contours('ROIs/' + i + '.nii.gz', levels=[1.], alpha = 1.0, 
                             colors='black', linewidths=educ_count[i]/3)
    
    print(i, educ_color[i])
    
#display.savefig('Results/Supp/Images/complexity_Educ_Brain.png', dpi=300)

In [ ]:
df_age = pd.DataFrame({'ROI': list(age_count.keys()), 
                   'Count age': list(age_count.values()), 
                   'Mean coef age': list(age_color.values())})

df_educ = pd.DataFrame({'ROI': list(educ_count.keys()),  
                   'Mean coef educ': list(educ_color.values())})

df_mmse = pd.DataFrame({'ROI': list(mmse_count.keys()), 
                   'Mean coef mmse': list(mmse_color.values())})

df_sex = pd.DataFrame({'ROI': list(sex_count.keys()), 
                   'Mean coef sex': list(sex_color.values())})

df = pd.merge(df_age, df_educ, on='ROI')
df = pd.merge(df, df_mmse, on='ROI')
df = pd.merge(df, df_sex, on='ROI')
df

## Total sample

In [ ]:
nested_lists = list(feat_for_test_without_mmse.values())

combined_list = []
for sublist in nested_lists:
    combined_list.extend(sublist)
    
combined_list


In [ ]:
compl_df_wo_mmse = pd.read_excel('Data/complexity_total_sample_supp.xlsx', index_col=0)
compl_df_wo_mmse

In [ ]:
dict_OLS_compl_coef_wo_mmse = {}
dict_OLS_compl_R2_wo_mmse = {}



for outcome in combined_list:

    y_compl_1A = compl_df_wo_mmse[outcome]
    X_compl_1A = compl_df_wo_mmse[['Age', 'Sex','Educ']]

    scaler = MinMaxScaler([0.5, 0.95])
    scaling_data = scaler.fit_transform(X_compl_1A)
    X_compl_1A_s = pd.DataFrame(scaling_data, columns= X_compl_1A.columns, index = X_compl_1A.index)
    
    X_compl_1A =  X_compl_1A_s.copy()


    res = summary_validation(X_compl_1A, y_compl_1A)
    table = res[0]
    table = table.iloc[1:-8, 0:-1]
    table = table[['Estimate mean', 't value mean', 'p value stouffer', 'Estimate std']]
    table = table.sort_values(by="Estimate mean", key=lambda x: abs(x), ascending=False)
    print(table.index[0])
    


    dict_OLS_compl_coef_wo_mmse[outcome] = res
    dict_OLS_compl_R2_wo_mmse[outcome] = res[6]
    
    print(outcome, res[6], X_compl_1A.shape[0])

In [ ]:
plt.figure(figsize=(7,9))
count=1


for keys in top_10_keys_Age[0:3]:
    plt.subplot(4,3,count)
    
    model_ = dict_OLS_compl_Age_coef[keys]
    
    #model_[0].to_excel('Results/Supp/Models/Complexity/compl_age_cog_' + keys + '.xlsx')
    #dict_OLS_compl_coef_wo_mmse[keys][0].to_excel('Results/Supp/Models/Complexity/compl_age_no_cog_' + keys + '.xlsx')

    table = model_[0]
    table = table.iloc[1:-8, 0:-1]
    table = table[['Estimate mean', 't value mean', 'p value stouffer', 'Estimate std']]
    table.index.name = 'Features'
    
    table = table.sort_values(by="Estimate mean", key=lambda x: abs(x), ascending=False)
    table = table.rename(columns= {'Estimate std':'std'})
    table = table.rename(columns= {'p value stouffer':'p_value'})
    table = table.rename(columns= {'Estimate mean':'coef'})
    
    table['coef'] = np.abs(table['coef'])

    max_v = table['coef'].max()
    max_v+=max_v*0.5

    if max_v>9 or max_v<3:
        pass
    else:
        max_v = 8
        
    plot_estimate_value_stat_model(table, model_,p_desp =0,  xlim =[0, max_v], title = keys,
                                   stat_model_WO_mmse = dict_OLS_compl_coef_wo_mmse[keys], pos = [0.3, 1.5])
    count+=1
    
    feat_for_test_without_mmse['Age'].append(keys)
    
for keys in range(3):
    plt.subplot(4,3,count)
    plt.xlabel('   ')
    plt.ylabel('    ')
    #plt.xticks([])
    #plt.yticks([])
    count+=1
    
for keys in top_10_keys_Educ[0:3]:
    plt.subplot(4,3,count)
    
    model_ = dict_OLS_compl_Educ_coef[keys]
    
    #model_[0].to_excel('Results/Supp/Models/Complexity/compl_educ_cog_' + keys + '.xlsx')
    #dict_OLS_compl_coef_wo_mmse[keys][0].to_excel('Results/Supp/Models/Complexity/compl_educ_no_cog_' + keys + '.xlsx')

    table = model_[0]
    table = table.iloc[1:-8, 0:-1]
    table = table[['Estimate mean', 't value mean', 'p value stouffer', 'Estimate std']]
    table.index.name = 'Features'
    
    table = table.sort_values(by="Estimate mean", key=lambda x: abs(x), ascending=False)
    table = table.rename(columns= {'Estimate std':'std'})
    table = table.rename(columns= {'p value stouffer':'p_value'})
    table = table.rename(columns= {'Estimate mean':'coef'})
    
    table['coef'] = np.abs(table['coef'])

    max_v = table['coef'].max()
    max_v+=max_v*0.5

    if max_v>9 or max_v<3:
        pass
    else:
        max_v = 8
        
    plot_estimate_value_stat_model(table, model_,p_desp =0,  xlim =[0, max_v], title = keys,
                                   stat_model_WO_mmse = dict_OLS_compl_coef_wo_mmse[keys], pos = [0.3, 1.5])
    count+=1
    
    feat_for_test_without_mmse['Educ'].append(keys)
    
for keys in range(3):
    plt.subplot(4,3,count)
    plt.xlabel('   ')
    plt.ylabel('    ')
    #plt.xticks([])
    #plt.yticks([])
    count+=1

count = 7
for keys in top_10_keys_MMSE[0:3]:
    plt.subplot(4,3,count)
    
    model_ = dict_OLS_compl_MMSE_coef[keys]
    
    #model_[0].to_excel('Results/Supp/Models/Complexity/compl_mmse_cog_' + keys + '.xlsx')
    #dict_OLS_compl_coef_wo_mmse[keys][0].to_excel('Results/Supp/Models/Complexity/compl_mmse_no_cog_' + keys + '.xlsx')

    table = model_[0]
    table = table.iloc[1:-8, 0:-1]
    table = table[['Estimate mean', 't value mean', 'p value stouffer', 'Estimate std']]
    table.index.name = 'Features'
    
    table = table.sort_values(by="Estimate mean", key=lambda x: abs(x), ascending=False)
    table = table.rename(columns= {'Estimate std':'std'})
    table = table.rename(columns= {'p value stouffer':'p_value'})
    table = table.rename(columns= {'Estimate mean':'coef'})
    
    table['coef'] = np.abs(table['coef'])

    max_v = table['coef'].max()
    max_v+=max_v*0.5
    
    if max_v>9:
        pass
    elif max_v< 0.015:
        max_v = 0.003
    elif max_v< 0.05:
        max_v = 0.05
    else:
        max_v = 2
    
    plot_estimate_value_stat_model(table, model_,p_desp =0,  xlim =[0, max_v], title = keys,
                                   stat_model_WO_mmse = dict_OLS_compl_coef_wo_mmse[keys], pos = [0.1, 1.5])
    count+=1
    
    feat_for_test_without_mmse['MMSE'].append(keys)

    

plt.tight_layout(pad=1.5);

#plt.savefig('Results/Supp/Images/complexity_Results.pdf', format='pdf')